In [4]:
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfTransformer

time: 329 µs (started: 2022-05-30 16:38:39 +00:00)


# Load data

In [5]:
with open("./datasets/test_leader_session.pkl" , "rb") as f:
    test_leader_dic = pickle.load(f)
    
# with open("./datasets/test_session.pkl" , "rb") as f:
#     test_dic = pickle.load(f)
    
# with open("./datasets/train_session.pkl" , "rb") as f:
#     train_dic = pickle.load(f)

# with open("./datasets/purchases_session.pkl" , "rb") as f:
#     purchases_dic = pickle.load(f)
    
# with open("./datasets/train_session_last_3m.pkl" , "rb") as f:
#     train_last_3_dic = pickle.load(f)
    
# with open("./datasets/purchases_last_3m.pkl" , "rb") as f:
#     purchases_last_3_dic = pickle.load(f)
    
with open("./datasets/train_session_last_1m.pkl" , "rb") as f:
    train_last_1_dic = pickle.load(f)
    
with open("./datasets/purchases_last_1m.pkl" , "rb") as f:
    purchases_last_1_dic = pickle.load(f)
    
test_leader_df = pd.DataFrame.from_dict(test_leader_dic, orient='index')
# test_df = pd.DataFrame.from_dict(test_dic, orient='index')
# train_df = pd.DataFrame.from_dict(train_dic, orient='index')
# purchases_df = pd.DataFrame.from_dict(purchases_dic, orient='index')
# train_last_3_df= pd.DataFrame.from_dict(train_last_3_dic, orient='index')
# purchases_last_3_df= pd.DataFrame.from_dict(purchases_last_3_dic, orient='index')
train_last_1_df= pd.DataFrame.from_dict(train_last_1_dic, orient='index')
purchases_last_1_df= pd.DataFrame.from_dict(purchases_last_1_dic, orient='index')
candidate_df = pd.read_csv("./datasets/candidate_vector.csv").set_index("item_id")

time: 22.7 s (started: 2022-05-30 16:38:41 +00:00)


In [ ]:
display(f"test_leader_df: {test_leader_df.shape}", test_leader_df.head())
display(test_leader_df.iloc[1,(test_leader_df.iloc[1,:]>0).tolist()], "-"*100)
display(f"test_df: {test_df.shape}", test_df.head(), "-"*100)
display(f"train_df: {train_df.shape}", train_df.head(), "-"*100)
display(f"purchases_df: {purchases_df.shape}", purchases_df.head(), "-"*100)
display(f"train_last_3_df: {train_last_3_df.shape}", train_last_3_df.head(), "-"*100)
display(f"purchases_last_3_df: {purchases_last_3_df.shape}", purchases_last_3_df.head(), "-"*100)
display(f"train_last_3_df: {train_last_1_df.shape}", train_last_1_df.head(), "-"*100)
display(f"purchases_last_3_df: {purchases_last_1_df.shape}", purchases_last_1_df.head(), "-"*100)
display(f"candidate: {candidate_df.shape}", candidate_df.head(), "-"*100)

## Concat raw data 

In [6]:
# tot_session_df =  pd.concat([test_leader_df, test_df, train_df, purchases_df])
# train_purchasees_last_3_df = pd.concat([train_last_3_df, purchases_last_3_df])
# train_last_3_leader_df = pd.concat([train_last_3_df, test_leader_df])
train_purchasees_last_1_df = pd.concat([train_last_1_df, purchases_last_1_df])
train_last_1_leader_df = pd.concat([train_last_1_df, test_leader_df])

time: 202 ms (started: 2022-05-30 16:39:04 +00:00)


In [ ]:
# print(f"session_df shape:{tot_session_df.shape}")
# display(tot_session_df.head())
display(f"train_last_3_leader_df: {train_last_3_leader_df.shape}", train_last_3_leader_df.head(), "-"*100)

# Build tfidf vector
- turn `session_df` to tfidf vector
- turn `candidte_df` to tfidf vector

In [7]:
model = TfidfTransformer()
model.fit(train_last_1_leader_df)
session_tfidf_sparse_mat = model.transform(test_leader_df)
session_tfidf_arr = session_tfidf_sparse_mat.toarray()
candidate_tfidf_sparse_mat = model.transform(candidate_df)
candidate_tfidf_arr = candidate_tfidf_sparse_mat.toarray()

time: 1.18 s (started: 2022-05-30 16:39:04 +00:00)


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:438: UserWarning: X has feature names, but TfidfTransformer was fitted without feature names
  warnings.warn(


In [8]:
print(f"candidate_tfidf_arr shape:{candidate_tfidf_arr.shape}")
display(candidate_tfidf_arr[0])
print(f"session_tfidf_arr shape:{session_tfidf_arr.shape}")
display(session_tfidf_arr[0])

candidate_tfidf_arr shape:(4990, 904)


array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

session_tfidf_arr shape:(50000, 904)


array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.34002581, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

time: 13.5 ms (started: 2022-05-30 16:39:05 +00:00)


# Compute similarity 
- similarity bewteen session_tfidf and candidate_tfidf
- rank by similarity

In [9]:
sim_arr = np.dot(session_tfidf_arr,candidate_tfidf_arr.T)
candidate_ri_df = candidate_df.reset_index()
argsort_sim_arr = np.argsort(sim_arr)[::,::-1]
item_id_rank_arr = np.apply_along_axis(lambda x:candidate_ri_df.loc[x[:100],"item_id"], 1, argsort_sim_arr).reshape(-1)

time: 21.2 s (started: 2022-05-30 16:39:05 +00:00)


In [10]:
print(sim_arr.shape, sim_arr[0].max(), argsort_sim_arr.shape, argsort_sim_arr[0].max())
display(sim_arr[0][argsort_sim_arr])
display(argsort_sim_arr)
display(item_id_rank_arr[:10])

(50000, 4990) 0.9999999999999998 (50000, 4990) 4989


array([[1.        , 0.95118555, 0.94595715, ..., 0.00641356, 0.00635346,
        0.        ],
       [0.17837469, 0.17837469, 0.17837469, ..., 0.016486  , 0.20647951,
        0.1160777 ],
       [0.22199095, 0.25020072, 0.21283331, ..., 0.00679333, 0.00641356,
        0.00635346],
       ...,
       [0.13661255, 0.12856476, 0.23411474, ..., 0.0282975 , 0.11763734,
        0.22358478],
       [0.12355119, 0.12355119, 0.19328346, ..., 0.01683085, 0.016486  ,
        0.006981  ],
       [0.26074931, 0.24824918, 0.26129745, ..., 0.00641356, 0.00635346,
        0.0688008 ]])

array([[3403, 2904,  605, ..., 3041, 3952, 3584],
       [ 271, 1091, 4208, ..., 2455, 1542,  283],
       [1483, 2288, 1180, ..., 2196, 3041, 3952],
       ...,
       [3385, 2517, 2762, ...,  413, 2611, 3013],
       [  41, 4572, 3019, ..., 3153, 2455, 2654],
       [ 404, 1548,  382, ..., 3041, 3952, 4682]])

array([19185, 16411,  3260, 17740, 22166, 18418, 23612,  7109,  5199,
       22736])

time: 790 ms (started: 2022-05-30 16:39:26 +00:00)


# Output results

In [11]:
target_df = test_leader_df
res_df = pd.DataFrame(np.repeat(target_df.index, 100), columns=["session_id"])
res_df["item_id"] = item_id_rank_arr
res_df["rank"] = (list(range(1,101))*len(target_df))
res_df.to_csv("./datasets/results/leader_tfidf_train_last_1_leader_0530.csv", index=False)

observe_ind = np.array([[i*100+j for j in range(10)] for i in range(len(target_df))]).reshape(-1)
res_df.iloc[observe_ind,:][:50]

,session_id,item_id,rank
0,26,19185,1
1,26,16411,2
2,26,3260,3
3,26,17740,4
4,26,22166,5
5,26,18418,6
6,26,23612,7
7,26,7109,8
8,26,5199,9
9,26,22736,10


time: 4.37 s (started: 2022-05-30 16:39:27 +00:00)
